In [2]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.4 MB/s eta 0:00

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import load_metric
import numpy as np
import os

In [4]:
os.chdir("/content/")

df1 = pd.read_csv("DanavisDFwithPadding-200.csv")
df2 = pd.read_csv("DanavisDFwithPadding-200:400.csv")
df3 = pd.read_csv("DanavisDFwithPadding-400:1000.csv")

df = pd.concat([df1, df2, df3])
df = df.dropna(subset=["label"])
df["label"] = df["label"].astype(int)
df = df.reset_index()[["comment_text", "label"]]
df

,comment_text,label
0,og eu-kommissionen mener at usa har,1
1,har halvårsresultatet for at det trak,1
2,at det trak sådan <pad> <pad>,0
3,vi havde alle lavet en parter,0
4,havde alle lavet en parter i,0
...,...,...
999995,svenska sportsbyg.rb-børsen romerdammen bliver...,0
999996,sportsbyg.rb-børsen romerdammen bliver ved med at,0
999997,romerdammen bliver ved med at sige,0
999998,med at sige at administrationen med,1


In [5]:
device = "cuda"
torch.device(device)

device(type='cuda')

In [6]:
#X = list(df["comment_text"])
#y = list(df["label"])
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1212)
#tokenizer = BertTokenizer.from_pretrained('Maltehb/danish-bert-botxo')
#X_train_tokenized = tokenizer(X_train, padding=True)
#max_x_tokenized = max([len(x) for x in X_train_tokenized])
#max_x_tokenized

14

In [7]:
X = list(df["comment_text"])
y = list(df["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1212)
tokenizer = BertTokenizer.from_pretrained('Maltehb/danish-bert-botxo')
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=15)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=15)

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, batch_size=16):
        self.encodings = encodings
        self.labels = labels
        self.batch_size = batch_size

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [14]:
batch_size = 32
epochs = 3

# Create the train and test datasets
train_dataset = CustomDataset(X_train_tokenized, y_train, batch_size=batch_size)
val_dataset = CustomDataset(X_val_tokenized, y_val, batch_size=batch_size)

# Create the model
model = BertForSequenceClassification.from_pretrained('Maltehb/danish-bert-botxo', num_labels=2)
#model = torch.load("modelCombined2")
model.to(device)


Some weights of the model checkpoint at Maltehb/danish-bert-botxo were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [17]:
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [21]:
torch.save(model, './commaModel3')

In [23]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.060000,0.095914,0.973520


{'eval_loss': 0.09591443091630936, 'eval_accuracy': 0.97352}